# Deploy Pet Breed Classifer on HuggingFace Space using Gradio

## Train and export Pet Breed Classifer using fastai

In [1]:
from fastai.vision.all import *

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
path = untar_data(URLs.PETS)

dls = ImageDataLoaders.from_name_re(
    path, 
    get_image_files(path/'images'), 
    pat='(.+)_\d+.jpg', 
    item_tfms=Resize(460), 
    batch_tfms=aug_transforms(size=224, min_scale=0.75))

learn = vision_learner(dls, models.resnet50, metrics=accuracy)
learn.fine_tune(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.962639,0.342378,0.886333,01:41


epoch,train_loss,valid_loss,accuracy,time
0,0.416534,0.287114,0.913396,01:39


In [15]:
img = PILImage.create('/kaggle/input/pet-breed-predict/samoyed.jpg')
pred,pred_idx,probs = learn.predict(img)
pred

'samoyed'

In [17]:
learn.path = Path('/kaggle/working')
learn.export('model.pkl')

## Gradio app

In [5]:
!pip install -Uqq gradio

In [24]:
import gradio as gr
from fastai.vision.all import *
import skimage

learn = load_learner('/kaggle/working/model.pkl')

labels = learn.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
    return dict(zip(labels, map(float, probs)))

title = "Pet Breed Classifier"
description = "A pet breed classifier trained on the Oxford Pets dataset with fastai."
article="<p style='text-align: center'><a href='https://github.com/mchen50' target='_blank'>My Github</a></p>"
examples = ['/kaggle/input/pet-breed-predict/siamese.jpg']
interpretation='default'

app = gr.Interface(
    fn=predict,
    inputs=gr.Image(shape=(512, 512)),
    outputs=gr.Label(num_top_classes=3),
    title=title,
    description=description,
    article=article,
    examples=examples,
    interpretation=interpretation)
app.queue()
app.launch()

Running on local URL:  http://127.0.0.1:7863
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://a5d1e748667d7a40ab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## HuggingFace Space API endpoint

In [1]:
!pip install -Uqq gradio_client

In [22]:
import requests
import json
from gradio_client import Client
from IPython.display import Image

image_url = "https://moderndogmagazine.com/sites/default/files/images/uploads/Inset-1_34.jpg"

client = Client("https://saicmsaicm-pet-breed.hf.space/")
result = client.predict(
    image_url,
    api_name="/predict"
)

with open(result, "r") as f:
    result_json = json.load(f)

print(
    f"The breed of this pet is a {(' '.join(result_json['label'].split('_')))}"
)

display(Image(url=image_url, width=475))

print("Original JSON returned from the request: ", json.dumps(result_json, indent=2))

Loaded as API: https://saicmsaicm-pet-breed.hf.space/ ✔
The breed of this pet is a shiba inu


Original JSON returned from the request:  {
  "label": "shiba_inu",
  "confidences": [
    {
      "label": "shiba_inu",
      "confidence": 0.9998903274536133
    },
    {
      "label": "samoyed",
      "confidence": 0.00010687798931030557
    },
    {
      "label": "pomeranian",
      "confidence": 6.84225881286693e-07
    }
  ]
}
